In [ ]:
import shutil, stat
import torch
import os
import glob
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
import numpy as np
from PIL import Image
from tqdm import tqdm
import math
%matplotlib inline

In [ ]:
module_base = r"C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\evaluation\\"
# MODULE 1 location of test_image and output
module_1_base = module_base + "module_1\\"
module_1_path = module_1_base + 'test_image.jpg'

# MODULE 2 location of test_image
module_2_base = module_base + "module_2\\"
module_2_path = module_2_base + 'test_image.jpg'

# MODULE 3 location of test_image
module_3_base = module_base + "module_3\\"
module_3_path = module_3_base + 'test_image.jpg'

# MODULE 4 location of test_image
module_4_base = module_base + "module_4\\"
module_4_path = module_4_base + 'test_image.jpg'

In [ ]:
def find_image_path(module):
    # location of test image
    if module == 1:
        image_path = module_1_path
    elif module == 2:
        image_path = module_2_path
    elif module == 3:
        image_path = module_3_path
    elif module == 4:
        image_path = module_4_path
        
    return image_path

In [ ]:
def find_module_path(module):
    
    if module == 1:
        module_path = 'module_1'
    elif module == 2:
        module_path = 'module_2'
    elif module == 3:
        module_path = 'module_3'
    elif module == 4:
        module_path = 'module_4'
    
    return module_path

In [ ]:
def cv_image_display(image, x_dim, y_dim):
    
    # resize to fit window
    y_sf = 0.3
    x_sf = 0.3
    y_dim_rs = int(y_dim*y_sf)
    x_dim_rs = int(x_dim*x_sf)
    image = cv.resize(image, (x_dim_rs, y_dim_rs)) 
    
    # display image
    cv.imshow('img', image)
    cv.waitKey(0)

In [ ]:
def hough(module, dp, min_dist, param_1, param_2, min_radius, max_radius, vis):
    
    image_path = find_image_path(module)
    output_path = '\\'.join(image_path.split('\\')[0:-1]) + '\\method_1_predicted\\'

#     print(image_path)
    img = cv.imread(image_path)
    y_dim = img.shape[0]
    x_dim = img.shape[1]

    gray = np.copy(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
    img = cv.medianBlur(gray,5)
    cimg = np.copy(cv.cvtColor(img,cv.COLOR_GRAY2BGR))


    circles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, dp, min_dist, 
                              param1=param_1, param2=param_2, 
                              minRadius=min_radius, maxRadius=max_radius)
    
    circles = np.uint16(np.around(circles))

    for i in circles[0,:]:
        # draw the outer circle
        cv.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        cv.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

    if vis:
        cv_image_display(cimg, x_dim, y_dim)
        

    xs = []
    ys = []

    for circle in circles[0]:
        x = int(circle[0])
        y = int(circle[1])

        xs.append(x)
        ys.append(y)
        
    return xs, ys

In [ ]:
def find_mm_px_r(module):
    
    # define pixel/mm ratio for the image

    if module == 1:
        mm_px_r = 18/239
        
    if module == 2:
        mm_px_r = 18/158
        
    if module == 3:
        mm_px_r = 18/217
        
    if module == 4:
        mm_px_r = 18/257
        
    return mm_px_r

In [ ]:
def find_delta(module, pred_x, pred_y, gt_path):
    
    with open(gt_path + '\\test_image' + '-x.json', "r") as fp:
        gt_x = json.load(fp)

    with open(gt_path + '\\test_image' + '-y.json', "r") as fp:
        gt_y = json.load(fp)
    
    
    mm_px_r = find_mm_px_r(module)
    
    # define mm threshold for a successful prediction
    min_delta_mm = 3

    delta_px = 0
    failed_gt_idx = []
    failed_pred_idx = []
    
    
    
    
    # find fails
    
    for i in range(len(gt_x)):

        smallest_delta = 1e6

        for j in range(len(pred_x)):
            # find the absolute difference in x
            x_delta = abs(gt_x[i] - pred_x[j])
            # find the absolute difference in y
            y_delta = abs(gt_y[i] - pred_y[j])
            # find euclidian distance of difference
            euclidian_delta = math.sqrt(x_delta**2 + y_delta**2)
            # keep the smallest distance
            if euclidian_delta < smallest_delta:
                prediction_index = j
                smallest_delta = euclidian_delta
                
        # flag if the smallest distance is greater than the threshold in mm
        if smallest_delta * mm_px_r > min_delta_mm:
            failed_gt_idx.append(i)
    
            if smallest_delta * mm_px_r < 10:
#                 print('predictions for ground truth point', i, 'failed by prediction', prediction_index)
                failed_pred_idx.append(prediction_index)
            else:
                failed_pred_idx.append(-1)
    

    
    
    # find px distance
    
    for j in range(len(pred_x)):
        smallest_delta = 1e6
        
        for i in range(len(gt_x)):
            # find the absolute difference in x
            x_delta = abs(gt_x[i] - pred_x[j])
            # find the absolute difference in y
            y_delta = abs(gt_y[i] - pred_y[j])
            # find euclidian distance of difference
            euclidian_delta = math.sqrt(x_delta**2 + y_delta**2)
            # keep the smallest distance
            if euclidian_delta < smallest_delta:
                smallest_delta = euclidian_delta
                
                # add the smallest distance to the total delta metric
        delta_px += smallest_delta
    
    delta_mm = delta_px * mm_px_r
    
    # difference in number of predicted spots to real spots
    delta_spots = abs(len(pred_x) - len(gt_x))
    
#     print('sum of errors in px=', delta_px,
#          '\nmissed/extra spots =', delta_spots,
#          '\nfails =', len(failed_gt_idx))
    
    return failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots

In [ ]:
# param_1_range = np.linspace(75,125,11)
# param_2_range = np.linspace(15,25,11)
param_1_range = np.linspace(10,400,11)
param_2_range = np.linspace(10,200,11)

module = 2
gt_path = f"C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\ground-truth"

delta_pxs = []
delta_mms = []
delta_spotss = []
fails = []

p1s = []
p2s = []

for p1 in tqdm(param_1_range):
    
    for p2 in param_2_range:
        
        try:

            pred_x, pred_y = hough(module, dp = 1, min_dist = 100, 
                                   param_1 = p1, param_2 = p2, 
                                   min_radius = 50, max_radius = 60, 
                                   vis=False)

            failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots = find_delta(module, pred_x, pred_y, gt_path)


            delta_pxs.append(delta_px)
            delta_mms.append(delta_mm)
            delta_spotss.append(delta_spots)
            fails.append(len(failed_gt_idx))

            p1s.append(p1)
            p2s.append(p2)
            
                
        except:
            print('error')

---

#### visualisations

In [ ]:
%matplotlib inline

X0, X1 = np.meshgrid(param_1_range, param_2_range)
levels = np.linspace(a.min(), a.max(), 30)

In [ ]:
t = []
for i in delta_spotss:
    if i != 0:
        t.append(math.log(i))
    else:
        t.append(i)
    
a = np.reshape(np.array(t), (11,11))
levels = np.linspace(a.min(), a.max(), 30)

fig, ax = plt.subplots()
CS = ax.contourf(X0, X1, a, levels=levels)
CB = fig.colorbar(CS)
ax.set_title('metric 1')
ax.set_xlabel('param_1')
ax.set_ylabel('param_2')

plt.show()

In [ ]:
t = [math.log(i) for i in delta_pxs]
a = np.reshape(np.array(t), (11,11))
levels = np.linspace(a.min(), a.max(), 30)

fig, ax = plt.subplots()
CS = ax.contourf(X0, X1, a, levels=levels)
CB = fig.colorbar(CS)
ax.set_title('metric 2')
ax.set_xlabel('param_1')
ax.set_ylabel('param_2')

plt.show()

In [ ]:
a = np.array(fails)
a = np.reshape(np.array(a), (11,11))
levels = np.linspace(a.min(), a.max(), 30)

fig, ax = plt.subplots()
CS = ax.contourf(X0, X1, a, levels=levels)
CB = fig.colorbar(CS)
ax.set_title('metric 3')
ax.set_xlabel('param_1')
ax.set_ylabel('param_2')

plt.show()

---

#### data and image analysis

In [ ]:
print(min(delta_spotss))

In [ ]:
print(min(delta_pxs))

In [ ]:
print(delta_spotss, delta_mms)

In [ ]:
t = delta_spotss.copy()
delta_spotss.sort()

idx = [t.index(i) for i in delta_spotss]

delta_spotss = t.copy()

In [ ]:
print(idx[0:5])
print(delta_spotss[0:5])

In [ ]:
delta_spotss

In [ ]:
import operator, collections

ds = {}
for k, v in enumerate(delta_spotss):
    ds[k] = v
    
sorted_ds = sorted(ds.items(), key=operator.itemgetter(1))
sorted_ds = dict(collections.OrderedDict(sorted_ds))

In [ ]:
dict(sorted_ds)

In [ ]:
for i in range(len(delta_spotss)):
    
#     if (delta_spotss[i] < 50) & (delta_pxs[i] < 10000):

        
        
    if (delta_mms[i] < 80) & (delta_spotss[i] < 80):

#     if delta_mms[i] < 40:


        print(i)
#         print(p1s[i], p2s[i])
        print('')


In [ ]:
i = 1
print(p1s[i], p2s[i])

pred_x, pred_y = hough(module, 
                       dp = 1, 
                       min_dist = 100, 
                       param_1 = p1s[i], param_2 = p2s[i], 
                       min_radius = 50, max_radius = 60,
                       vis=True)

gt_path = f"C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\ground-truth"

failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots = find_delta(module, pred_x, pred_y, gt_path)

print('delta_px =', delta_px, 
      '\ndelta_mm =',delta_mm, 
      '\ndelta_spots =',delta_spots,
      '\nfails =', len(failed_gt_idx))

In [ ]:
module = 2
pred_x, pred_y = hough(module, 
                       dp = 1, 
                       min_dist = 100, 
                       param_1 = 115, param_2 = 22, 
                       min_radius = 50, max_radius = 60,
                       vis=True)

gt_path = f"C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\ground-truth"

failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots = find_delta(module, pred_x, pred_y, gt_path)

print('delta_px =', delta_px, 
      '\ndelta_mm =',delta_mm, 
      '\ndelta_spots =',delta_spots,
      '\nfails =', len(failed_gt_idx))

---

#### top-performers

In [ ]:
class HoughParams:
    
    def __init__(self, dp, min_dist, param_1, param_2, min_radius, max_radius):
        self.dp = dp
        self.min_dist = min_dist
        self.param_1 = param_1
        self.param_2 = param_2
        self.min_radius = min_radius
        self.max_radius = max_radius
        
module_1_params = HoughParams(dp = 1, min_dist = 100, param_1 = 180, param_2 = 19, min_radius = 115, max_radius = 130)
module_2_params = HoughParams(dp = 1, min_dist = 100, param_1 = 115, param_2 = 22, min_radius = 50, max_radius = 60)
module_3_params = HoughParams(dp = 1, min_dist = 100, param_1 = 85, param_2 = 30, min_radius = 105, max_radius = 120)
module_4_params = HoughParams(dp = 1, min_dist = 100, param_1 = 30, param_2 = 30, min_radius = 123, max_radius = 135)

In [ ]:
module = 2

if module == 1:
    params = module_1_params
elif module == 2:
    params = module_2_params
elif module == 3: 
    params = module_3_params
elif module == 4:
    params = module_4_params

pred_x, pred_y = hough(module, 
                       dp = params.dp, 
                       min_dist = params.min_dist, 
                       param_1 = params.param_1, param_2 = params.param_2, 
                       min_radius = params.min_radius, max_radius = params.max_radius,
                       vis = True)

gt_path = f"C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\ground-truth"

failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots = find_delta(module, pred_x, pred_y, gt_path)

print('delta_px =', delta_px, 
      '\ndelta_mm =',delta_mm, 
      '\ndelta_spots =',delta_spots,
      '\nfails =', len(failed_gt_idx))

In [ ]:
image_path = find_image_path(module)
output_path = '\\'.join(image_path.split('\\')[0:-1]) + '\\method_1_predicted\\'

im = mpimg.imread(image_path)
fig = plt.imshow(im)
plt.scatter(pred_x, pred_y, c='r', marker="x", alpha=0.7, linewidth=0.5)
plt.show
plt.savefig(output_path + 'test_image' + '.png', dpi=500)

with open(output_path + 'test_image' + '-x.json', "w") as fp:
    json.dump(pred_x, fp)

with open(output_path + 'test_image' + '-y.json', "w") as fp:
    json.dump(pred_y, fp)

In [ ]:
pred_x, pred_y = hough(module, 
                       dp = params.dp, 
                       min_dist = params.min_dist, 
                       param_1 = params.param_1, param_2 = params.param_2, 
                       min_radius = params.min_radius, max_radius = params.max_radius,
                       vis = True)

---